In [18]:
import pandas as pd
import numpy as np

In [46]:
# Each file on flight data represents one month of data
FlightDataDates = ['2017.08',
                   '2017.09',
                   '2017.10',
                   '2017.11',
                   '2017.12',
                   '2018.01',
                   '2018.02',
                   '2018.03',
                   '2018.04',
                   '2018.05',
                   '2018.06',
                   '2018.07']

#FlightDataDates = ['2017.08',
#                   '2017.09']

In [28]:
# Read the data file and strip out the unused columns
# DataDate - string in the form '2018.04'
# Return a cleaned DataFrame

def ReadFlightData(DataDate):
    df_read = pd.read_csv('data/ONTIME_'+DataDate+'.csv')
    df = df_read[['FL_DATE','OP_UNIQUE_CARRIER','OP_CARRIER_FL_NUM','ORIGIN_AIRPORT_ID',
                  'DEST_AIRPORT_ID','CRS_ARR_TIME','ARR_TIME','DISTANCE','CANCELLED',
                  'CARRIER_DELAY','WEATHER_DELAY','NAS_DELAY', 'SECURITY_DELAY',
                  'LATE_AIRCRAFT_DELAY']]
    return df

In [47]:
df = pd.DataFrame()
for DataDate in FlightDataDates:
    print(DataDate)
    df = df.append(ReadFlightData(DataDate))
              

2017.08
2017.09
2017.10
2017.11
2017.12
2018.01
2018.02
2018.03
2018.04
2018.05
2018.06
2018.07


In [48]:
#df = ReadFlightData('2017.09')
df.head()

,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,CRS_ARR_TIME,ARR_TIME,DISTANCE,CANCELLED,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,2017-08-12,B6,497,10721,14747,2308,2302.0,2496,0,NaN,NaN,NaN,NaN,NaN
1,2017-08-12,B6,498,14747,10721,818,813.0,2496,0,NaN,NaN,NaN,NaN,NaN
2,2017-08-12,B6,597,10721,14747,1018,951.0,2496,0,NaN,NaN,NaN,NaN,NaN
3,2017-08-12,B6,598,14747,10721,1934,1925.0,2496,0,NaN,NaN,NaN,NaN,NaN
4,2017-08-12,B6,907,14747,10299,2318,2257.0,1448,0,NaN,NaN,NaN,NaN,NaN


In [51]:
df.shape

(306810, 14)

In [49]:
sorted(df['OP_UNIQUE_CARRIER'].unique())

['AA', 'AS', 'B6', 'DL', 'F9', 'G4', 'HA', 'NK', 'OO', 'UA', 'VX', 'WN']

In [4]:
pd.read_csv('data/ChicagoWeather.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (25,44,45,63,64,67,84,85,86,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,STATION,STATION_NAME,ELEVATION,LATITUDE,LONGITUDE,DATE,REPORTTPYE,HOURLYSKYCONDITIONS,HOURLYVISIBILITY,HOURLYPRSENTWEATHERTYPE,...,MonthlyMaxSeaLevelPressureTime,MonthlyMinSeaLevelPressureValue,MonthlyMinSeaLevelPressureDate,MonthlyMinSeaLevelPressureTime,MonthlyTotalHeatingDegreeDays,MonthlyTotalCoolingDegreeDays,MonthlyDeptFromNormalHeatingDD,MonthlyDeptFromNormalCoolingDD,MonthlyTotalSeasonToDateHeatingDD,MonthlyTotalSeasonToDateCoolingDD
0,WBAN:94846,CHICAGO OHARE INTERNATIONAL AIRPORT IL US,201.8,41.995,-87.9336,2017-01-01 00:51,FM-12,CLR:00,10.00,NaN,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
1,WBAN:94846,CHICAGO OHARE INTERNATIONAL AIRPORT IL US,201.8,41.995,-87.9336,2017-01-01 01:51,FM-15,CLR:00,10.00,NaN,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
2,WBAN:94846,CHICAGO OHARE INTERNATIONAL AIRPORT IL US,201.8,41.995,-87.9336,2017-01-01 02:51,FM-15,CLR:00,10.00,NaN,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
3,WBAN:94846,CHICAGO OHARE INTERNATIONAL AIRPORT IL US,201.8,41.995,-87.9336,2017-01-01 03:51,FM-15,CLR:00,8.00,NaN,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
4,WBAN:94846,CHICAGO OHARE INTERNATIONAL AIRPORT IL US,201.8,41.995,-87.9336,2017-01-01 04:51,FM-15,CLR:00,8.00,NaN,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
5,WBAN:94846,CHICAGO OHARE INTERNATIONAL AIRPORT IL US,201.8,41.995,-87.9336,2017-01-01 05:51,FM-15,CLR:00,8.00,NaN,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
6,WBAN:94846,CHICAGO OHARE INTERNATIONAL AIRPORT IL US,201.8,41.995,-87.9336,2017-01-01 06:00,FM-12,NaN,NaN,NaN,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
7,WBAN:94846,CHICAGO OHARE INTERNATIONAL AIRPORT IL US,201.8,41.995,-87.9336,2017-01-01 06:51,FM-15,CLR:00,7.00,NaN,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
8,WBAN:94846,CHICAGO OHARE INTERNATIONAL AIRPORT IL US,201.8,41.995,-87.9336,2017-01-01 07:51,FM-15,CLR:00,8.00,NaN,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
9,WBAN:94846,CHICAGO OHARE INTERNATIONAL AIRPORT IL US,201.8,41.995,-87.9336,2017-01-01 08:51,FM-15,CLR:00,8.00,NaN,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
